# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [37]:
output_data = pd.read_csv("../output_data/cities.csv")
output_data

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodinsk,58.607673,99.177909,-0.65,88,79,9.51,RU,1612932782
1,1,sorvag,62.071293,-7.305866,30.20,74,82,3.44,FO,1612932782
2,2,iralaya,14.997464,-83.225170,76.37,81,12,8.01,HN,1612932782
3,3,isangel,-19.541720,169.280350,82.54,66,2,12.15,VU,1612932783
4,4,okato,-39.191082,173.879984,64.00,76,96,8.99,NZ,1612932783
...,...,...,...,...,...,...,...,...,...,...
63,65,port alfred,-33.586407,26.885145,70.00,95,1,2.84,ZA,1612932792
64,66,saint-philippe,-21.358118,55.764656,84.20,54,0,9.22,RE,1612932792
65,67,lockhart,29.884944,-97.670000,52.00,76,90,11.50,US,1612932792
66,68,nantucket,41.283470,-70.099451,33.01,93,90,14.97,US,1612932765


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
gmaps.configure(api_key=g_key)

In [39]:
locations = output_data[["Latitude", "Longitude"]].astype(float)
humidity = output_data['Humidity'].astype(float)

In [40]:
fig = gmaps.figure(center=(10,20), zoom_level=.7)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
ideal_output = output_data.loc[((output_data['Max Temp'] > 40) & 
                                (output_data['Max Temp'] < 80) &
                                (output_data['Humidity'] < 70))]
ideal_output

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,san cristobal,16.736993,-92.637599,78.80,61,40,17.27,MX,1612932783
9,9,busselton,-33.653158,115.345489,69.01,57,2,5.99,AU,1612932783
17,17,bluff,37.284445,-109.551788,42.80,49,100,14.97,US,1612932785
26,26,coihaique,-45.571225,-72.068265,62.60,63,0,3.44,CL,1612932786
27,27,ushuaia,-54.801912,-68.302951,64.40,68,0,11.50,AR,1612932786
37,38,bubaque,11.283333,-15.833333,76.78,63,33,12.10,GW,1612932787
67,69,barstow,34.895796,-117.017283,60.80,29,1,11.50,US,1612932548


In [42]:
ideal_locations = ideal_output[["Latitude", "Longitude"]].astype(float)
ideal_humidity = ideal_output['Humidity'].astype(float)

In [43]:
new_fig = gmaps.figure(center=(10,20), zoom_level=.7)

ideal_heat_layer = gmaps.heatmap_layer(ideal_locations, weights=ideal_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

new_fig.add_layer(ideal_heat_layer)

new_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
ideal_output['Hotel Name'] = ""
ideal_output

C:\Users\16129\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,san cristobal,16.736993,-92.637599,78.80,61,40,17.27,MX,1612932783,
9,9,busselton,-33.653158,115.345489,69.01,57,2,5.99,AU,1612932783,
17,17,bluff,37.284445,-109.551788,42.80,49,100,14.97,US,1612932785,
26,26,coihaique,-45.571225,-72.068265,62.60,63,0,3.44,CL,1612932786,
27,27,ushuaia,-54.801912,-68.302951,64.40,68,0,11.50,AR,1612932786,
37,38,bubaque,11.283333,-15.833333,76.78,63,33,12.10,GW,1612932787,
67,69,barstow,34.895796,-117.017283,60.80,29,1,11.50,US,1612932548,


In [46]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_output.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row['City Name']
    country = row['Country']

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#    print(name_address.url)

    # convert to json
    hotel = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ideal_output.loc[index, "Hotel Name"] = hotel['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
ideal_output

C:\Users\16129\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,san cristobal,16.736993,-92.637599,78.80,61,40,17.27,MX,1612932783,Hotel Provincia
9,9,busselton,-33.653158,115.345489,69.01,57,2,5.99,AU,1612932783,Busselton Beach Resort
17,17,bluff,37.284445,-109.551788,42.80,49,100,14.97,US,1612932785,Bluff Dwellings Resort & Spa
26,26,coihaique,-45.571225,-72.068265,62.60,63,0,3.44,CL,1612932786,Abriga Bed and Breakfast
27,27,ushuaia,-54.801912,-68.302951,64.40,68,0,11.50,AR,1612932786,Patagonia Jarke
37,38,bubaque,11.283333,-15.833333,76.78,63,33,12.10,GW,1612932787,BOB FISHING CLUB BIJAGOS
67,69,barstow,34.895796,-117.017283,60.80,29,1,11.50,US,1612932548,Best Western Desert Villa Inn


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_output.iterrows()]
locations = ideal_output[["Latitude", "Longitude"]]

In [49]:
# Add marker layer ontop of heat map
hotel_symbol_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)', 
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5, 
    info_box_content=[f"{hotel}" for hotel in hotel_info])


hotel_fig = gmaps.figure()
hotel_fig.add_layer(hotel_symbol_layer)

hotel_fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# Display Figure
combined_fig = gmaps.figure()

combined_fig.add_layer(ideal_heat_layer)
combined_fig.add_layer(hotel_symbol_layer)

combined_fig


Figure(layout=FigureLayout(height='420px'))